In [1]:
import numpy as np
import tensorflow as tf
import math
import cv2
import matplotlib.pyplot as plt
import time
import VGG16

In [2]:
def l_oneshot(label,c):
    arr=np.zeros((label[:,:,0].shape[0]*label[:,:,0].shape[1],c),np.uint8)
    flat_label=label[:,:,0].flatten()
    for x in range(len(flat_label)):
        arr[x,flat_label[x]]=1
    arr=np.reshape(arr,(label[:,:,0].shape[0],label[:,:,0].shape[1],c))
    return arr

def im_resize(img,size):
#     x=np.minimum(img.shape[0],img.shape[1])
#     re_img=img[0:x,0:x]
    im=cv2.resize(img, (size,size), interpolation=cv2.INTER_LINEAR)
    return im
def label_resize(img,size):
#     x=np.minimum(img.shape[0],img.shape[1])
#     re_img=img[0:x,0:x]
    im=cv2.resize(img, (size,size), interpolation=cv2.INTER_NEAREST)
    return im

In [3]:
def depth_resize(img,size):
#     x=np.minimum(img.shape[0],img.shape[1])
#     re_img=img[0:x,0:x]
    im=np.reshape(cv2.resize(img, (size,size), interpolation=cv2.INTER_LINEAR),(size,size,1))
    return im

In [4]:
def d_resize(img,size):
#     x=np.minimum(img.shape[0],img.shape[1])
#     re_img=img[0:x,0:x]
    im=cv2.resize(img, (size,size), interpolation=cv2.INTER_LINEAR)
    return im

In [5]:
def l_oneshot_14(label):
    arr=np.zeros((label.shape[0]*label.shape[1],14),np.uint8)
    flat_label=label.flatten()
    for x in range(len(flat_label)):
        arr[x,flat_label[x]]=1
    arr=np.reshape(arr,(label.shape[0],label.shape[1],14))
    return arr

In [6]:
def bn_relu(layer,is_use):
    mean,var=tf.nn.moments(layer,[1,2])
    num=tf.to_int32(mean.shape[1])
    mean_1=tf.reshape(mean,[-1,1,1,num])
    var_1=tf.reshape(var,[-1,1,1,num])
    layer_1=(layer-mean_1)/var_1
    return tf.nn.relu(layer_1)

In [7]:
def bn_elu(layer, use_bn):
    mean,var=tf.nn.moments(layer,[1,2])
    exp_mean=tf.expand_dims(mean,1)
    exp_var=tf.expand_dims(var,1)
    exp_mean_1=tf.expand_dims(exp_mean,2)
    exp_var_1=tf.expand_dims(exp_var,2)
    layer_1=tf.nn.elu(layer-exp_mean_1)
    return layer_1

In [8]:
def act_func_c(layer,use_bn):
    layer_1=tf.nn.relu(layer)
    return layer_1

In [9]:
def act_func_d(layer,use_bn):
    layer_1=tf.maximum(layer,layer*0.3)
    return layer_1

In [10]:
def norm_relu(layer):  
    mean,var=tf.nn.moments(layer,[1])
    mean_1=tf.reshape(mean,[-1,1])
    var_1=tf.reshape(var,[-1,1])
    layer_1=(layer-mean_1)/var_1
    return tf.nn.relu(layer_1)

In [11]:
def make_wei(name,ker_s,in_l_num,out_l_num):
    w= tf.get_variable('w_'+name, shape=[ker_s,ker_s,in_l_num,out_l_num], initializer=tf.contrib.layers.xavier_initializer_conv2d())
    b=tf.get_variable('b_'+name, shape=[out_l_num], initializer=tf.contrib.layers.xavier_initializer())
    return w,b

In [12]:
def conv_relu(layer,is_use):
    mean,var=tf.nn.moments(layer,[1,2])
    num=tf.to_int32(mean.shape[1])
    mean_1=tf.reshape(mean,[-1,1,1,num])
    var_1=tf.reshape(var,[-1,1,1,num])
    layer_1=(layer-mean_1)/var_1
    return tf.concat([tf.nn.relu(layer_1),tf.nn.relu(-1.0*layer_1)],3)

In [13]:
def conv_relu_sig(layer,is_use):
    return tf.concat([tf.nn.relu(layer),tf.nn.sigmoid(layer)],3)

In [14]:
def get_conv(in_layer,k_s,in_n,out_n,l_n,name,use_bn):
    in_l=in_layer
    for i in range(l_n):
        if i==0:
            w,b=make_wei(name+'_'+str(i+1),k_s,in_n,out_n)
            in_l=conv_relu(tf.nn.conv2d(in_l,w, strides=[1,1,1,1], padding= 'SAME')+b,use_bn)
        else:
            w,b=make_wei(name+'_'+str(i+1),k_s,out_n*2,out_n)
            in_l=conv_relu(tf.nn.conv2d(in_l,w, strides=[1,1,1,1], padding= 'SAME')+b,use_bn)
    return in_l

In [15]:
def get_2_conv(in_layer,k_s,in_n,out_n,l_n,name,use_bn):
    in_l=in_layer
    for i in range(l_n):
        if i==0:
            w,b=make_wei(name+'_'+str(i+1),k_s,in_n,out_n)
            in_l=conv_relu_sig(tf.nn.conv2d(in_l,w, strides=[1,1,1,1], padding= 'SAME')+b,use_bn)
        else:
            w,b=make_wei(name+'_'+str(i+1),k_s,out_n*2,out_n)
            in_l=conv_relu_sig(tf.nn.conv2d(in_l,w, strides=[1,1,1,1], padding= 'SAME')+b,use_bn)
    return in_l

In [16]:
def get_c(in_layer,k_s,in_n,out_n,l_n,name,use_bn):
    in_l=in_layer
    for i in range(l_n):
        if i==0:
            w,b=make_wei(name+'_'+str(i+1),k_s,in_n,out_n)
            in_l=norm_relu(tf.nn.conv2d(in_l,w, strides=[1,1,1,1], padding= 'SAME')+b,use_bn)
        else:
            w,b=make_wei(name+'_'+str(i+1),k_s,out_n*2,out_n)
            in_l=norm_relu(tf.nn.conv2d(in_l,w, strides=[1,1,1,1], padding= 'SAME')+b,use_bn)
    return in_l

In [17]:
def get_fc(in_layer,k_s,in_n,out_n,l_n,name,use_bn):
    in_l=in_layer
    for i in range(l_n):
        if i==0:
            a=in_n
        else:
            a=out_n
        w,b=make_wei(name+'_'+str(i+1),k_s,a,out_n)
        in_l=bn_relu(tf.nn.conv2d(in_l,w, strides=[1,1,1,1], padding= 'SAME')+b,use_bn)
    return in_l

In [18]:
def get_edgemap(in_layer,k_s,in_n,out_n,name,use_bn):
    w_1,b_1=make_wei(name+'_1',k_s,in_n,out_n)
    in_1=bn_relu(tf.nn.conv2d(in_layer,w_1, strides=[1,1,1,1], padding= 'SAME')+b_1,use_bn)
    
    w,b=make_wei(name+'_2',k_s,out_n,out_n)
    out_l=bn_relu(tf.nn.conv2d(in_1,w, strides=[1,1,1,1], padding= 'SAME')+b,use_bn)
    return out_l

In [19]:
def make_fc(name,in_l_num,out_l_num,in_layer):
    w= tf.get_variable('w_'+name, shape=[in_l_num,out_l_num], initializer=tf.contrib.layers.xavier_initializer())
    b=tf.get_variable('b_'+name, shape=[out_l_num], initializer=tf.contrib.layers.xavier_initializer())
    
    return tf.matmul(in_layer,w)+b

In [20]:
def seg_deconv(in_l,k_s,i_s,edge,edge_n,in_n,out_n,name,is_train):
    w_edge_1,b_edge_1=make_wei(name+'_1',k_s,in_n+edge_n,in_n)
    w_edge_2,b_edge_2=make_wei(name+'_2',k_s,in_n,in_n)
    w_edge_3,b_edge_3=make_wei(name+'_3',k_s,in_n,out_n)
    

 
    edge_1=tf.image.resize_images(in_l,[math.ceil(i_s),math.ceil(i_s)],method=tf.image.ResizeMethod.BILINEAR)
    edge_2=tf.concat([edge,edge_1],3)
    edge_3=bn_relu(tf.nn.conv2d(edge_2,w_edge_1, strides=[1,1,1,1], padding= 'SAME')+b_edge_1,is_train)
    edge_4=bn_relu(tf.nn.conv2d(edge_3,w_edge_2, strides=[1,1,1,1], padding= 'SAME')+b_edge_2,is_train)
    edge_5=bn_relu(tf.nn.conv2d(edge_4,w_edge_3, strides=[1,1,1,1], padding= 'SAME')+b_edge_3,is_train)


    return edge_5

In [21]:
def dep_deconv(in_l,k_s,i_s,edge,edge_n,in_n,out_n,name,is_train):
    w_edge_1,b_edge_1=make_wei(name+'_1',k_s,in_n+edge_n,in_n)
    w_edge_2,b_edge_2=make_wei(name+'_2',k_s,in_n,in_n)
    w_edge_3,b_edge_3=make_wei(name+'_3',k_s,in_n,out_n)
    

 
    edge_1=tf.image.resize_images(in_l,[math.ceil(i_s),math.ceil(i_s)],method=tf.image.ResizeMethod.BILINEAR)
    edge_2=tf.concat([edge,edge_1],3)
    edge_3=bn_relu(tf.nn.conv2d(edge_2,w_edge_1, strides=[1,1,1,1], padding= 'SAME')+b_edge_1,is_train)
    edge_4=bn_relu(tf.nn.conv2d(edge_3,w_edge_2, strides=[1,1,1,1], padding= 'SAME')+b_edge_2,is_train)
    edge_5=bn_relu(tf.nn.conv2d(edge_4,w_edge_3, strides=[1,1,1,1], padding= 'SAME')+b_edge_3,is_train)


    return edge_5

In [22]:
def iou_mean(inter,union,label,ch):
    m_iou=0
    m_mean=0
    none_zero=np.ones_like(union)
    label=np.maximum(label,none_zero)
    union=np.maximum(union,none_zero)
    
    s_iou=inter/union
    s_mean=inter/label

    
    for m in range(ch):
        m_iou+=s_iou[m]
        m_mean+=s_mean[m]
    print("mean_iou_acc : ", '{:.4f}'.format(m_iou/ch) )       
    print("mean_mean_acc : ", '{:.4f}'.format(m_mean/ch))
    return s_iou,s_mean,m_iou/ch,m_mean/ch

In [23]:
def acc(output,label,c):
    inter=output*label
    union_1=output+label
    union_2=tf.ones_like(union_1)
    union=tf.minimum(union_1,union_2)
    
    union_sum=tf.reduce_sum(union,0)
    label_sum=tf.reduce_sum(label,0)
    inter_sum=tf.reduce_sum(inter,0)

    return inter_sum, label_sum,union_sum

In [24]:
def dep_deri(dep_img,d_de):
    d_x_deri = tf.reshape(d_de, [3, 3, 1, 1])
    d_y_deri = tf.transpose(d_x_deri, [1, 0, 2, 3])
    

    d_x_deri=tf.nn.conv2d(dep_img, d_x_deri, strides=[1, 1, 1, 1], padding='VALID')
    d_y_deri=tf.nn.conv2d(dep_img, d_y_deri, strides=[1, 1, 1, 1], padding='VALID')

    
    d_x_loss=tf.square(d_x_deri/2.0)
    d_y_loss=tf.square(d_y_deri/2.0)
    
        
    dep_x_cost=tf.reshape(d_x_loss,[-1,int(d_x_loss.shape[1])*int(d_x_loss.shape[2])])
    dep_y_cost=tf.reshape(d_y_loss,[-1,int(d_y_loss.shape[1])*int(d_y_loss.shape[2])])
    dep_deri_cost=tf.reduce_mean(tf.reduce_mean(dep_x_cost,1)+tf.reduce_mean(dep_y_cost,1))
    
    return dep_deri_cost

In [37]:
def get_crf(img,seg_img,dep_img,deri,num):
    s_img=tf.square(tf.nn.atrous_conv2d(seg_img, deri,rate=num, padding='VALID'))
    
    im=tf.exp((-1*0.05*tf.square(tf.nn.atrous_conv2d(img, deri,rate=num, padding='VALID')))-(0.01*(num-1)))
    
    s_i_crf=tf.multiply(s_img,im)
    s_d_crf=tf.multiply((1-s_img),(1-im))
    s_sum=(tf.reduce_mean(tf.reduce_sum(s_i_crf,1))*0.5)+(tf.reduce_mean(tf.reduce_sum(s_d_crf,1))*0.5)
    
    d_img=tf.exp((-1*0.05*tf.square(tf.nn.atrous_conv2d(dep_img, deri,rate=num, padding='VALID')))-(0.01*(num-1)))
    
    d_i_crf=tf.multiply(s_img,d_img)
    d_d_crf=tf.multiply((1-s_img),(1-d_img))
    d_sum=(tf.reduce_mean(tf.reduce_sum(d_i_crf,1))*0.5)+(tf.reduce_mean(tf.reduce_sum(d_d_crf,1))*0.5)
    
    crf_sum=(s_sum*0.6)+(d_sum*0.4)
    return crf_sum

In [26]:
def seg_deri(seg_img,dep_img,img,ch,s_d_1,s_d_2,s_d_3,s_d_4):
    s_deri_1 = tf.reshape(s_d_1, [3, 3, 1, 1])
    s_deri_2 = tf.reshape(s_d_2, [3, 3, 1, 1])
    s_deri_3 = tf.reshape(s_d_3, [3, 3, 1, 1])
    s_deri_4 = tf.reshape(s_d_4, [3, 3, 1, 1])
    seg_crf=0
    
    for n in range(ch):
        seg_deri=tf.slice(seg_img,[0,0,0,n],[-1,-1,-1,1])
        for i in range(2):
            seg_crf_1=get_crf(img,seg_deri,dep_img,s_deri_1,i+1)
            seg_crf_2=get_crf(img,seg_deri,dep_img,s_deri_2,i+1)
            seg_crf_3=get_crf(img,seg_deri,dep_img,s_deri_3,i+1)
            seg_crf_4=get_crf(img,seg_deri,dep_img,s_deri_4,i+1)
            seg_crf+=(seg_crf_1+seg_crf_2+seg_crf_3+seg_crf_4)/4.0
    return seg_crf/ch

In [27]:
size=224
train_n=10000
test_n=3000

In [28]:
train_label=[label_resize(cv2.imread('img_data/train_label/img-'+str(i+1).zfill(6)+'.png',0),size) for i in range(train_n)]#5285

In [29]:
train_image=[im_resize(cv2.imread('img_data/train_image/img-'+str(i+1).zfill(6)+'.jpg'),size) for i in range(train_n)]#5285

In [30]:
train_depth=[depth_resize(cv2.imread('img_data/train_depth/img-'+str(i+1).zfill(6)+'.png',0),size) for i in range(train_n)]#5285

In [31]:
test_label=[label_resize(cv2.imread('img_data/test_label/img-'+str(i+1).zfill(6)+'.png',0),size) for i in range(test_n)]

In [32]:
test_image=[im_resize(cv2.imread('img_data/test_image/img-'+str(i+1).zfill(6)+'.jpg'),size) for i in range(test_n)]#5050

In [33]:
test_depth=[depth_resize(cv2.imread('img_data/test_depth/img-'+str(i+1).zfill(6)+'.png',0),size) for i in range(test_n)]#5285

In [34]:
h=size
w=size
c=14

In [35]:
print(h,w,c)

224 224 14


In [36]:
version='18'

In [52]:
for i in range(train_n):
    plt.imsave('dep_seg_'+version+'/output_img-'+str(i+1).zfill(6)+'_st.png',label_resize(cv2.imread('img_data/train_label/img-'+str(i+1).zfill(6)+'.png',0),size))
for i in range(test_n):
    plt.imsave('dep_seg_'+version+'/test_image/test_img-'+str(i+1).zfill(6)+'_st.png',label_resize(cv2.imread('img_data/test_label/img-'+str(i+1).zfill(6)+'.png',0),size))
# for i in range(train_n):
#     plt.imsave('dep_seg_'+version+'/output_img-'+str(i+1).zfill(6)+'_dt.png',d_resize(cv2.imread('img_data/train_depth/img-'+str(i+1).zfill(6)+'.png',0),size))
# for i in range(test_n):
#     plt.imsave('dep_seg_'+version+'/test_image/test_img-'+str(i+1).zfill(6)+'_dt.png',d_resize(cv2.imread('img_data/test_depth/img-'+str(i+1).zfill(6)+'.png',0),size))

In [38]:
training_epochs=400
tf.reset_default_graph()
x=tf.placeholder(tf.float32,[None,h,w,3]) 
s=tf.placeholder(tf.float32, [None,h,w])
d=tf.placeholder(tf.float32, [None,h,w,1])
keep_prob=tf.placeholder(tf.float32)
train_type=tf.placeholder(tf.bool)
global_step = tf.placeholder(tf.float32)
seg_deri_1=tf.constant([[0,0,0],[0,1,-1],[0,0,0]], tf.float32)
seg_deri_2=tf.constant([[0,-1,0],[0,1,0],[0,0,0]], tf.float32)
seg_deri_3=tf.constant([[0,0,-1],[0,1,0],[0,0,0]], tf.float32)
seg_deri_4=tf.constant([[0,0,0],[0,1,0],[0,0,-1]], tf.float32)

In [39]:
l_d_1_3=get_conv(d,3,1,16,2,'d_1',train_type)
l_d_1_3=tf.layers.dropout(l_d_1_3,rate=keep_prob)
l_d_1=tf.nn.max_pool(l_d_1_3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


l_d_2_3=get_2_conv(l_d_1,3,32,16,2,'d_2',train_type)
l_d_2_3=tf.layers.dropout(l_d_2_3,rate=keep_prob)
l_d_2=tf.nn.max_pool(l_d_2_3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


l_d_3_3=get_2_conv(l_d_2,3,32,16,3,'d_3',train_type)
l_d_3_3=tf.layers.dropout(l_d_3_3,rate=keep_prob)
l_d_3=tf.nn.max_pool(l_d_3_3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


l_d_4_3=get_2_conv(l_d_3,3,32,16,3,'d_4',train_type)
l_d_4_3=tf.layers.dropout(l_d_4_3,rate=keep_prob)

In [40]:
l_c_1_3=get_conv(x,2,3,32,2,'c_1',train_type)
l_c_1_2=tf.concat([l_c_1_3,l_d_1_3],3)
l_c_1=tf.nn.max_pool(l_c_1_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
l_c_1=tf.layers.dropout(l_c_1,rate=keep_prob)

l_c_2_3=get_2_conv(l_c_1,2,64+32,128,2,'c_2',train_type)
l_c_2_2=tf.concat([l_c_2_3,l_d_2_3],3)
l_c_2=tf.nn.max_pool(l_c_2_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
l_c_2=tf.layers.dropout(l_c_2,rate=keep_prob)

l_c_3_3=get_2_conv(l_c_2,3,256+32,256,3,'c_3',train_type)
l_c_3_2=tf.concat([l_c_3_3,l_d_3_3],3)
l_c_3=tf.nn.max_pool(l_c_3_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
l_c_3=tf.layers.dropout(l_c_3,rate=keep_prob)

l_c_4_3=get_2_conv(l_c_3,3,512+32,512,3,'c_4',train_type)
l_c_4_2=tf.concat([l_c_4_3,l_d_4_3],3)
l_c_4=tf.nn.max_pool(l_c_4_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
l_c_4=tf.layers.dropout(l_c_4,rate=keep_prob)

l_c_5_3=get_2_conv(l_c_4,3,1024+32,512,4,'c_5',train_type)
l_c_5=tf.layers.dropout(l_c_5_3,rate=keep_prob)

In [41]:
l_c_4_a=get_edgemap(l_c_4_2,3,1024+32,128,'c_4_a', train_type)
l_c_4_a=tf.layers.dropout(l_c_4_a,rate=keep_prob)

l_c_3_a=get_edgemap(l_c_3_2,3,512+32,128,'c_3_a', train_type)
l_c_3_a=tf.layers.dropout(l_c_3_a,rate=keep_prob)

l_c_2_a=get_edgemap(l_c_2_2,3,256+32,64,'c_2_a', train_type)
l_c_2_a=tf.layers.dropout(l_c_2_a,rate=keep_prob)

l_c_1_a=get_edgemap(l_c_1_2,3,64+32,64,'c_1_a', train_type)
l_c_1_a=tf.layers.dropout(l_c_1_a,rate=keep_prob)

In [42]:
l_seg_5=get_2_conv(l_c_5,1,1024,512,3,'seg_5',train_type)
l_seg_5=tf.layers.dropout(l_seg_5,rate=keep_prob)

l_seg_5_s=tf.image.resize_images(l_seg_5,[math.ceil(h/8),math.ceil(h/8)],method=tf.image.ResizeMethod.BILINEAR)
l_seg_5_s=tf.concat([l_seg_5_s,l_c_4_a],3)
l_seg_4=get_2_conv(l_seg_5_s,3,1024+128,256,3,'seg_4',train_type)
l_seg_4=tf.layers.dropout(l_seg_4,rate=keep_prob)

l_seg_4_s=tf.image.resize_images(l_seg_4,[math.ceil(h/4),math.ceil(h/4)],method=tf.image.ResizeMethod.BILINEAR)
l_seg_4_s=tf.concat([l_seg_4_s,l_c_3_a],3)
l_seg_3=get_2_conv(l_seg_4_s,3,512+128,128,3,'seg_3',train_type)
l_seg_3=tf.layers.dropout(l_seg_3,rate=keep_prob)

l_seg_3_s=tf.image.resize_images(l_seg_3,[math.ceil(h/2),math.ceil(h/2)],method=tf.image.ResizeMethod.BILINEAR)
l_seg_3_s=tf.concat([l_seg_3_s,l_c_2_a],3)
l_seg_2=get_2_conv(l_seg_3_s,3,256+64,128,3,'seg_2',train_type)



l_seg_2_s=tf.image.resize_images(l_seg_2,[math.ceil(h),math.ceil(h)],method=tf.image.ResizeMethod.BILINEAR)
l_seg_2_s=tf.concat([l_seg_2_s,l_c_1_a],3)
l_seg_1=get_2_conv(l_seg_2_s,3,256+64,64,3,'seg_1',train_type)

In [43]:
l_seg_0=get_fc(l_seg_1,5,128, 128,3,'seg_0',train_type)

In [44]:
w_seg_f_1,b_seg_f_1=make_wei('seg_f_1',3,128,c)

In [45]:
l_seg_f=tf.nn.conv2d(l_seg_0,w_seg_f_1, strides=[1,1,1,1], padding= 'SAME')+b_seg_f_1

In [46]:
flat_f_output=tf.reshape(l_seg_f,[-1,c])

s_f_int=tf.to_int64(s)

flat_s_f=tf.reshape(s_f_int,[-1,c])

In [47]:
s_one_f=tf.one_hot(s_f_int,c)
l_seg_sig=tf.nn.sigmoid(l_seg_f)
l_seg_avg_3=tf.nn.avg_pool(l_seg_sig, ksize=[1,3,3,1], strides=[1,1,1,1], padding='SAME')
l_seg_last=(l_seg_sig*0.4)+(l_seg_avg_3*0.6)
label_1=tf.nn.avg_pool(s_one_f, ksize=[1,3,3,1], strides=[1,1,1,1], padding='SAME')
label=(s_one_f*0.4)+(label_1*0.6)

In [48]:
seg_cost=tf.nn.sigmoid_cross_entropy_with_logits(logits=l_seg_f,labels=label)
seg_cost=tf.reshape(seg_cost,[-1,size*size*c])
seg_cost=tf.reduce_mean(seg_cost,1)
seg_cost=tf.reduce_mean(seg_cost)

In [49]:
s_image_1=tf.argmax(l_seg_last,3)
s_c_image=tf.nn.sigmoid(l_seg_f)

In [50]:
seg_one_hot=tf.one_hot(s_image_1,c)
f_seg_o=tf.reshape(seg_one_hot,[-1,c])
f_s_o=tf.reshape(s_one_f,[-1,c])

inter,label,union=acc(f_seg_o,f_s_o,c)

In [51]:
l_f_5=tf.reshape(tf.nn.max_pool(l_c_5_3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME'),[-1,7*7*1024])

In [52]:
l_f_5_1 = tf.nn.relu(make_fc('f_5',7*7*1024,512,l_f_5))
l_fc_1 = tf.nn.relu(make_fc('fc_1',512,256,l_f_5_1))
l_fc_2 = make_fc('fc_2',256,c,l_fc_1)

In [53]:
s_pro=tf.reshape(seg_one_hot,[-1,h*w,c])
s_pro=tf.div(tf.reduce_sum(s_pro,1),(h*w))+0.0001
fc_soft=tf.nn.softmax(l_fc_2)+0.0001
fc_cost=tf.reduce_sum(fc_soft*tf.log(tf.div(fc_soft,s_pro)))

In [54]:
# fc_cost=tf.nn.sigmoid_cross_entropy_with_logits(logits=l_fc_2,labels=s_pro)
# fc_cost=tf.reduce_sum(fc_cost,1)
# fc_cost=tf.reduce_mean(fc_cost)

In [55]:
gray=tf.image.rgb_to_grayscale(x)
crf_cost=seg_deri(l_seg_sig,d,gray,c,seg_deri_1,seg_deri_2,seg_deri_3,seg_deri_4)

In [56]:
# cost=seg_cost+dep_cost+dep_seg_cost*0.1
cost=(seg_cost*0.5)+(fc_cost*0.3)+(crf_cost*0.2)

In [57]:
starter_learning_rate = 0.0001
learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,5, 0.9, staircase=True)
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [ ]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())
saver=tf.train.Saver()
# saver.restore(sess,"dep_seg_"+version+"/seg_model.ckpt")
print("Learning start")
for epoch in range(training_epochs):
    a=0
    batch_size=2
    total_batch =math.ceil(train_n/batch_size)
    start=time.time()
    avg_cost=0
    avg_s_cost=0
    
    avg_i_cost=0

    avg_inter=np.zeros((c))
    avg_gt=np.zeros((c))
    avg_union=np.zeros((c))


    for i in range(total_batch):
        if a+batch_size<train_n:
            batch_xs=train_image[a:a+batch_size]
            batch_ys=train_label[a:a+batch_size]
            batch_ds=train_depth[a:a+batch_size]

        
        feed_dict={x:batch_xs, s:batch_ys, d:batch_ds, keep_prob:0.5, train_type:True, global_step:epoch}

        rate,in_1,gt_1,un_1,s_c,total_c,_=sess.run([learning_rate,inter,label,union,seg_cost,cost,optimizer], feed_dict=feed_dict)
        s_c_img,s_img_1=sess.run([s_c_image, s_image_1], feed_dict=feed_dict)


        for j in range(len(batch_xs)):
            plt.imsave('dep_seg_'+version+'/output_img-'+str(a+j+1).zfill(6)+'_se.png',s_img_1[j])
    
        for k in range(batch_size):
            for q in range(c):
                plt.imsave('dep_seg_'+version+'/t/class_img_'+str(a+k+1)+'_'+str(q).zfill(2)+'.png',s_c_img[k,:,:,q])

        a=a+batch_size
        avg_cost+=total_c
        avg_s_cost+=s_c


        avg_inter+=in_1
        avg_gt+=gt_1
        avg_union+=un_1



    print("epoch : ", epoch)
    print("cost : ", '{:.5f}'.format(avg_cost/total_batch),"seg_cost : ", '{:.5f}'.format(avg_s_cost/total_batch))
    s_iou_1,s_mean_1,m_iou_1,m_mean_1=iou_mean(avg_inter,avg_union,avg_gt,c)
    
    saver.save(sess, "dep_seg_"+version+"/seg_model.ckpt")
    end=time.time()
    print(end-start)
    print("\n")
    
    
    if epoch%2==0:
        test_a=0
        test_batch_size=2
        test_total_batch = math.ceil(test_n/test_batch_size)
        test_avg_cost=0
        test_avg_s_cost=0

        
        test_avg_inter_1=np.zeros((c))
        test_avg_gt_1=np.zeros((c))
        test_avg_union_1=np.zeros((c))

        t_start=time.time()
        for i in range(test_total_batch):
            if test_a+test_batch_size<test_n:
                test_batch_xs=test_image[test_a:test_a+test_batch_size]
                test_batch_ys=test_label[test_a:test_a+test_batch_size]
                test_batch_ds=test_depth[test_a:test_a+test_batch_size]

            test_feed_dict={x:test_batch_xs, s:test_batch_ys, d:test_batch_ds, keep_prob:1.0,train_type:False,global_step:epoch}
            t_in_1,t_gt_1,t_un_1,t_s_c,t_c,t_s_img_1=sess.run([inter,label,union, seg_cost,cost,s_image_1], feed_dict=test_feed_dict)
            for k in range(len(test_batch_xs)):
                plt.imsave('dep_seg_'+version+'/test_image/test_img-'+str(test_a+k+1).zfill(6)+'_se_1.png',t_s_img_1[k])
                
            test_a=test_a+test_batch_size
            test_avg_cost+=t_c
            test_avg_s_cost+=t_s_c
            

            test_avg_inter_1+=t_in_1
            test_avg_gt_1+=t_gt_1
            test_avg_union_1+=t_un_1
            
        print("----------------------------test_network----------------------------")
        print("cost : ",'{:.5f}'.format(test_avg_cost/test_total_batch), "seg_cost : ",'{:.5f}'.format(test_avg_s_cost/test_total_batch))
        test_s_iou_1,test_s_mean_1,test_m_iou_1,test_m_mean_1=iou_mean(test_avg_inter_1,test_avg_union_1,test_avg_gt_1,c)
        t_end=time.time()
        print(t_end-t_start)
        print("\n")

    if epoch%4==0:
        print("train_iou")
        print(np.round(s_iou_1,3))
        print("train_mean")
        print(np.round(s_mean_1,3))
        print("\n")
        print("test_iou")
        print(np.round(test_s_iou_1,3))
        print("test_mean")
        print(np.round(test_s_mean_1,3))
        print("\n")
        print(rate)
        print("\n")
        feed_dict={x:batch_xs, s:batch_ys, d:batch_ds, keep_prob:1.0, train_type:True, global_step:epoch}
        seg_img_1,seg_img_2,seg_img_3,seg_img_4=sess.run([l_c_1_a,l_c_2_a,l_c_3_a,l_c_4_a], feed_dict=feed_dict)
        
        for q in range(64):
            plt.imsave('dep_seg_'+version+'/edge_map/edge_map_1_'+str(q).zfill(2)+'.png',seg_img_1[0,:,:,q])
            plt.imsave('dep_seg_'+version+'/edge_map/edge_map_2_'+str(q).zfill(2)+'.png',seg_img_2[0,:,:,q])
            plt.imsave('dep_seg_'+version+'/edge_map/edge_map_3_'+str(q).zfill(2)+'.png',seg_img_3[0,:,:,q])
            plt.imsave('dep_seg_'+version+'/edge_map/edge_map_4_'+str(q).zfill(2)+'.png',seg_img_4[0,:,:,q])

Learning start
epoch :  0
cost :  17.03071 seg_cost :  0.73770
mean_iou_acc :  0.1055
mean_mean_acc :  0.1700
7361.820220708847


----------------------------test_network----------------------------
cost :  17.87916 seg_cost :  0.88594
mean_iou_acc :  0.0867
mean_mean_acc :  0.1646
397.3456230163574


train_iou
[ 0.258  0.123  0.003  0.011  0.1    0.404  0.06   0.015  0.005  0.023
  0.049  0.001  0.407  0.018]
train_mean
[ 0.49   0.186  0.005  0.017  0.136  0.655  0.075  0.017  0.007  0.029
  0.059  0.002  0.678  0.023]


test_iou
[ 0.139  0.     0.001  0.001  0.134  0.38   0.     0.     0.     0.     0.114
  0.001  0.438  0.005]
test_mean
[ 0.182  0.     0.001  0.001  0.483  0.877  0.     0.     0.     0.     0.199
  0.001  0.554  0.005]


0.0001




In [55]:
print(rate)

0.000228768
